***번개장터 데이터 가져오기***

In [ ]:
def concat_bungae_files():
    base_df = pd.DataFrame()
    for idx in range(9):
        df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')
        df = df.dropna(axis=0)
        df['cat_id'] = df['cat_id'].astype(int).astype(str)
        # -- 필요한 컬럼만 가져오기 -- 
        df = df[['product_id', 'product_name', 'cat_id']].copy()
        # -- concat 할 때마다 base_df 업데이트 -- 
        base_df = pd.concat([base_df, df], ignore_index=True)

    # -- base_df 드라이브에 저장해놓기 --
    # base_df.to_csv('/content/drive/MyDrive/bungae_base_df_fashion.csv', index=False)

    return base_df


sample_df = concat_bungae_files()
sample_df


# 베이스 코드

In [ ]:
# -- 예시로 여성 패딩 카테고리만 가져와보기 --
# 310090050 롱패딩
# 310090060 숏패딩
# 310090020 블루종/항공점퍼
df = sample_df[sample_df['cat_id'].isin(['310090050', '310090060', '310090020'])].copy()

In [ ]:
# -- 전처리 --
# -- 기호 없애기 --
text = 'bcbg 정품 오리털패딩(택포)'
import string
def remove_punct(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    text = text.lower().translate(remove_punct_dict)
    return text

df['product_name'] = df['product_name'].apply(lambda x : remove_punct(x))

In [ ]:
df = df.reset_index(drop=True)

***Word2Vec, FastText 활용하지 않기***

In [ ]:
"""활용 X"""
# from gensim.models import Word2Vec
# from gensim.models.fasttext import FastText
# from konlpy.tag import Mecab
# import gensim

**숏패딩**

In [ ]:
short = df[df['cat_id'] == '310090060'].copy()
short = short.reset_index(drop=True).copy()
short.head()

In [ ]:
print(len(short))

In [ ]:
short_filtered = short[~short['product_name'].str.contains('롱|항공|블루종')].copy()
short_filtered = short_filtered['product_name'][short_filtered['product_name'].str.contains('숏')].copy()
print(len(short_filtered))
short_filtered.head(10)

**롱패딩**

In [ ]:
long = df[df['cat_id'] == '310090050'].copy()
long = long.reset_index(drop=True).copy()
long.head()

In [ ]:
long_filtered = long[~long['product_name'].str.contains('숏|항공|블루종')].copy()
long_filtered = long_filtered['product_name'][long_filtered['product_name'].str.contains('롱')].copy()
print(len(long_filtered))
long_filtered.head(20)

**항공점퍼|블루종**

In [ ]:
hangong = df[df['cat_id'] == '310090020'].copy()
hangong = hangong.reset_index(drop=True).copy()
hangong.head()

In [ ]:
hangong_filtered = hangong[~hangong['product_name'].str.contains('숏|롱')].copy()
hangong_filtered = hangong_filtered['product_name'][hangong_filtered['product_name'].str.contains('항공|블루종')].copy()
print(len(hangong_filtered))
hangong_filtered.head(20)

In [ ]:
"""-- 여기까지 여성의류 > 패딩 > 숏패딩, 롱패딩, 항공점퍼|블루종 예시 --"""

# 전체 카테고리에 대한 코드

In [ ]:
import json
from collections import defaultdict
import string

def concat_bungae_files():
    base_df = pd.DataFrame()
    for idx in range(9):
        df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')
        df = df.dropna(axis=0)
        df['cat_id'] = df['cat_id'].astype(int).astype(str)
        # -- 필요한 컬럼만 가져오기 -- 
        df = df[['product_id', 'product_name', 'cat_id']].copy()
        # -- concat 할 때마다 base_df 업데이트 -- 
        base_df = pd.concat([base_df, df], ignore_index=True)

    # -- base_df 드라이브에 저장해놓기 --
    # base_df.to_csv('/content/drive/MyDrive/bungae_base_df_fashion.csv', index=False)

    return base_df

# -- 전처리 --
# -- 기호 없애기 --
def remove_punct(text):
    remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
    text = text.lower().translate(remove_punct_dict)
    return text

def bring_fashion_cat_csv():
    """
    번개장터 카테고리 id와 카테고리 이름이 들어있는
    파일 가져오는 함수
    """
    final_cat = pd.read_csv('./final_category.csv')
    final_cat = final_cat.drop('Unnamed: 0', axis=1).copy()
    final_cat['cat_id'] = final_cat['cat_id'].astype(str)
    
    return final_cat

def get_cat_name(cat_id:str) -> str:
    """
    카테고리 id를 입력하면 카테고리 이름을 반환하는 함수 => 중분류에 대해서
    """
    final_cat = bring_fashion_cat_csv()
    if len(cat_id) == 6: # 중분류
        return list(final_cat[final_cat['cat_id'] == cat_id]['cat2'])[0]
    elif len(cat_id) == 9: # 대분류
        return list(final_cat[final_cat['cat_id'] == cat_id]['cat3'])[0]

# -- subs 가 있는 것들만 dictionary로 뽑아내기 --
def make_category_dict_with_subs() -> dict:
    
    with open('./bgzt_fashion_category_nums.json', 'r') as file:
        data = json.load(file)
        
    category_dict_with_subs = dict()
    for main, mids in data.items():
        for mid, subs in mids.items():
            d = defaultdict()
            if subs != [None]: # 하위 sub들이 있을 때 
                d[mid] = subs
            category_dict_with_subs.update(d)
    return category_dict_with_subs


# -- 중분류 내에서의 소분류 morphs 키워드 가져오는 함수 --
def make_subcat_morphs_by_midcat(cat_dict:dict) -> dict:
    """
    make_category_dict_with_subs 함수로 부터 받은 딕션어리로
    midcat의 하위 subcat들의 morphs 키워드 가져오는 함수 
    """
    mecab = Mecab()
    d = defaultdict(list)

    for mid, subs in cat_dict.items():
        tmp_dict = dict()
        for sub in subs:

            mid_morph = mecab.morphs(get_cat_name(mid))
            sub_morph = mecab.morphs(remove_punct(get_cat_name(sub)))
            unique_sub_morph = set(sub_morph) - set(mid_morph)
            unique_sub_morph = list(unique_sub_morph)
            if unique_sub_morph == []:
                d[mid] += sub_morph
            d[mid] += unique_sub_morph

    return dict(d)

# -- 위의 두 함수 합치기 --
def get_dict_from_midcat_with_subs():
    cat_dict = make_category_dict_with_subs()
    morphs_dict = make_subcat_morphs_by_midcat(cat_dict)
    return morphs_dict


# -- 다음으로는 mid만 있는 것들만 뽑아내기 --
def make_category_lists_without_subs():
    """
    소분류 카테고리가 없는 중분류 리스트 => cat_ids_without_subs
    """
    # -- 패션 cat_id 들어있는 json 파일 가져오기 --
    with open('./bgzt_fashion_category_nums.json', 'r') as file:
        data = json.load(file)

    cat_list_without_subs = list()
    morphs_list = list()
    for main, mids in data.items():
        for mid in mids:
            if mids[mid] == [None]:
                morphs_list += mecab.morphs(remove_punct(get_cat_name(mid)))
                cat_list_without_subs.append(mid)
    
    morphs_list = list(set(morphs_list))
    
    return morphs_list, cat_list_without_subs

In [ ]:
# -- 카테고리 번호가 9인, 즉 sub 카테고리가 있는 상품들에 대해 
def make_df_for_nine_nums_cat_ids():
    sample_df = concat_bungae_files()
    morphs_dict_with_subs = get_dict_from_midcat_with_subs()
    
    final_df = pd.DataFrame()
    for cat_id in list(sample_df['cat_id'].unique()):
        if len(cat_id) == 9:
            try:
                tmp_df = sample_df[sample_df['cat_id'] == cat_id]
                cat_id_morphs = mecab.morphs(remove_punct(get_cat_name(cat_id)))
                mid_id_name = mecab.morphs(remove_punct(get_cat_name(cat_id[:6])))
                midcat_morphs = morphs_dict_with_subs[cat_id[:6]]

                cat_id_morphs = [morph for morph in cat_id_morphs if morph in midcat_morphs]
                other_morphs = [morph for morph in midcat_morphs if morph not in cat_id_morphs]
                opposite_keywords = '|'.join(other_morphs)

                tmp_df_filtered = tmp_df[~tmp_df['product_name'].str.contains(opposite_keywords)]

                if '기타' in cat_id_morphs:
                    df = tmp_df_filtered
                else:
                    if '긴팔' in cat_id_morphs:
                        cat_id_morphs += ['롱']
                    elif '반팔' in cat_id_morphs:
                        cat_id_morphs += ['숏']    
                    else:
                        cat_id_morphs = cat_id_morphs
                    df = tmp_df_filtered[tmp_df_filtered['product_name'].str.contains('|'.join(cat_id_morphs))]
                    
                final_df = pd.concat([final_df, df], ignore_index=True)
            except:
                pass
        
    return final_df

def make_df_for_six_num_ids():
    morphs_list_without_subs, cat_list_without_subs = make_category_lists_without_subs()
    
    final_df = pd.DataFrame()
    for cat_id in list(sample_df['cat_id'].unique()):
        if cat_id in cat_list_without_subs:
            
            tmp_df = sample_df[sample_df['cat_id'] == cat_id].copy()
            cat_id_morphs = mecab.morphs(remove_punct(get_cat_name(cat_id)))

            mid_id_name = mecab.morphs(remove_punct(get_cat_name(cat_id[:6])))
            other_morphs = [morph for morph in morphs_list_without_subs if morph not in cat_id_morphs]
            opposite_keywords = '|'.join(other_morphs)

            tmp_df_filtered = tmp_df[~tmp_df['product_name'].str.contains(opposite_keywords)].copy()

            if '기타' in cat_id_morphs:
                df = tmp_df_filtered
            else:
                df = tmp_df_filtered[tmp_df_filtered['product_name'].str.contains('|'.join(cat_id_morphs))].copy()
                
            final_df = pd.concat([final_df, df])
    
    return final_df


def concat_two_df(): 
    final_df1 = make_df_for_nine_nums_cat_ids()
    final_df2 = make_df_for_six_num_ids()
    df = pd.concat([final_df1, final_df2], ignore_index=True)
    return df # 대략 반개 줄어들음 

final_df = concat_two_df()
len(final_df)

In [ ]:
import matplotlib.pyplot as plt

final_df['product_name_length'] = final_df['product_name'].apply(lambda x : len(x))
final_df

In [ ]:
plt.hist(final_df['product_name_length'], bins=50)
plt.show()

In [ ]:
final_df['cat_id'].value_counts().median()

In [ ]:
cat_id_count = final_df['cat_id'].value_counts()
plt.bar(height = cat_id_count, x= cat_id_count.index)

# FastText로 진행해보기?

- FastText를 통해서 진행을 했을때 관련이 없는 단어들이 나오긴 했지만 <br>
그래도 해당 단어들로 데이터들을 모았을 때 대략 (데이터가 너무 적은 카테고리를 빼고) <br>
500~3000 사이로 들어오는 것 같음
- 그래서 해당 방법으로 데이터를 모아보기
- 하나의 가설로 해보기

In [ ]:
import gc

def get_df_by_fasttext(cat_id:str):
    df = sample_df[sample_df['cat_id'] == cat_id]
    df = df.reset_index(drop=True)
    
    # -- tokenizing --
    tokenizer = Mecab()

    tokens = []
    for idx in range(len(df)):
        token = tokenizer.morphs(df.loc[idx]['product_name'])
        if len(token) > 1:
            tokens.append(token)
            
    # -- fasttext training --       
    for _ in range(20):
        model2 = FastText(tokens, vector_size=100, window=10, min_count=5, workers=4, sg=0)
        model2.build_vocab(tokens)
        total_examples = model2.corpus_count
        model2.train(tokens, total_examples=model2.corpus_count, epochs=10)
        
    # -- get most similar words -- 
    results = model2.wv.most_similar(positive=['스키니', '진'], negative=['일자', '부츠', '컷', '배기', '기타'], topn=20)
    keywords_list = [result[0] for result in results]
    keywords = str('|'.join(keywords_list))
    print(keywords)
    
    
    # -- get df --
    df_filtered = df[df['product_name'].str.contains(keywords)]
    
    del model2
    gc.collect()
    
    return df_filtered

In [ ]:
get_df_by_fasttext('310140010')

In [ ]:
for keyword in keywords_list:
    for token in tokens:
        if keyword in token:
            print(keyword, token)
    

***전처리 후 진행***

In [ ]:
# -- 전처리 함수 -- 
import re

def remove_punct(text: str) -> str:
    remove_punct_dict = dict((ord(punct), ' ') for punct in string.punctuation)
    text = text.lower().translate(remove_punct_dict)
    return text

def remove_english_and_numbers(text: str) -> str:
    text = re.sub('[a-zㄱ-ㅎ0-9]', '', text).strip()
    text = re.sub('\s{2,}', ' ', text)
    return text

def preprocessing_product_name(product_name: str) -> str:
    product_name = remove_punct(product_name)
    product_name = remove_english_and_numbers(product_name)
    return product_name

In [ ]:
fashion_df = sample_df.copy()

In [ ]:
fashion_df['product_name'] = fashion_df['product_name'].apply(lambda x : preprocessing_product_name(x))
fashion_df.head()

In [ ]:
fashion_df.sample(30)

In [ ]:
def get_df_by_fasttext_test_version(cat_id:str, positive:list, negative:list):
    df = fashion_df[fashion_df['cat_id'] == cat_id]
    df = df.reset_index(drop=True)
    
    # -- tokenizing --
    tokenizer = Mecab()

    tokens = []
    for idx in range(len(df)): # 명사인 토큰만 가져오기 
        token = [pos[0] for pos in mecab.pos(df.loc[idx]['product_name']) if pos[1][0] == 'N']            
        if len(token) > 1:
            tokens.append(token)
            
    # -- fasttext training --       
    for _ in range(20):
        model2 = FastText(tokens, vector_size=100, window=10, min_count=5, workers=4, sg=0)
        model2.build_vocab(tokens)
        total_examples = model2.corpus_count
        model2.train(tokens, total_examples=model2.corpus_count, epochs=10)
        
    # -- get most similar words -- 
    results = model2.wv.most_similar(positive=positive, negative=negative, topn=20)
    keywords_list = [result[0] for result in results]
    keywords = str('|'.join(keywords_list))
    print(keywords)
    
    
    # -- get df --
    df_filtered = df[df['product_name'].str.contains(keywords)]
    print(len(df_filtered))
    
    del model2
    gc.collect()
    
    return df_filtered.sample(30)

get_df_by_fasttext_test_version('320140400', ['정장', '세트'], ['자켓', '베스트', '바지', '기타'])

In [ ]:
def get_df_by_fasttext_test_version2(cat_id:str, positive:list, negative:list):
    df = fashion_df[fashion_df['cat_id'] == cat_id]
    df = df.reset_index(drop=True)
    
    # -- tokenizing --
    tokenizer = Mecab()

    tokens = []
    for idx in range(len(df)): # 명사인 토큰만 가져오기 
        token = [pos[0] for pos in mecab.pos(df.loc[idx]['product_name']) if pos[1][0] == 'N']            
        if len(token) > 1:
            tokens.append(token)
            
    # -- model 1 training --       
    model1 = FastText(tokens, vector_size=100, window=10, min_count=5, workers=4, sg=0)
    model1.build_vocab(tokens)
    total_examples1 = model1.corpus_count
    model1.train(tokens, total_examples=total_examples1, epochs=10)
    
    # -- model 2 training --
    model2 = FastText(tokens, vector_size=100, window=10, min_count=5, workers=4, sg=0)
    model2.build_vocab(tokens)
    total_examples2 = model2.corpus_count
    model2.train(tokens, total_examples=total_examples2, epochs=10)
        
    # -- get most similar words -- 
    results1 = model1.wv.most_similar(positive=positive, topn=20)
    results2 = model2.wv.most_similar(positive=negative, topn=20)
    
    keywords_list1 = [result[0] for result in results1]
    keywords_list2 = [result[0] for result in results2]
    
    keywords1 = str('|'.join(keywords_list1))
    keywords2 = str('|'.join(keywords_list2))
    
    print(keywords1)
    print(keywords2)
    
    
    # -- get df --
    df1 = df[df['product_name'].str.contains(keywords1)]
    df2 = df[df['product_name'].str.contains(keywords2)]
    idx_list = list(set(df1.index) - set(df2.index))
    df_filtered = df.iloc[idx_list]
    print(len(df_filtered))
    
    del model1, model2
    gc.collect()
    
    return df_filtered

get_df_by_fasttext_test_version2('320110500', ['배기', '청바지'], ['일자', '스키니', '진', '부츠', '컷','기타'])

In [ ]:
def get_df_by_fasttext_test_version3(cat_id:str, positive:list, negative=None):
    df = fashion_df[fashion_df['cat_id'] == cat_id]
    df = df.reset_index(drop=True)
    
    # -- tokenizing --
    tokenizer = Mecab()

    tokens = []
    for idx in range(len(df)): # 명사인 토큰만 가져오기 
        token = [pos[0] for pos in mecab.pos(df.loc[idx]['product_name']) if pos[1][0] == 'N']            
        if len(token) > 1:
            tokens.append(token)
            
    # -- model 1 training --       
    model1 = FastText(tokens, vector_size=100, window=10, min_count=5, workers=4, sg=0)
    model1.build_vocab(tokens)
    total_examples1 = model1.corpus_count
    model1.train(tokens, total_examples=total_examples1, epochs=10)
    
    # -- get most similar words -- 
    results1 = model1.wv.most_similar(positive=positive, topn=20)
    keywords_list1 = [result[0] for result in results1]
    keywords1 = str('|'.join(keywords_list1))
    print(keywords1)
    
    # -- 반대되는 데이터 프레임 --
    if negative != None:
        opposite_keywords = '|'.join(negative)
        df = df[~df['product_name'].str.contains(opposite_keywords)].copy()
    
    # -- get df --
    df_filtered = df[df['product_name'].str.contains(keywords1)]
    print(len(df_filtered))
    
    del model1
    gc.collect()
    
    return df_filtered

get_df_by_fasttext_test_version3('310110030', ['겨울', '코트'], ['봄', '가을'])

In [ ]:
mecab.morphs('맨투맨')

# 버전 3로 진행

해당 작업 목표를 다시 상기하면 <br>
- 데이터가 너무 많아서 모델 학습 시간이 오래 걸려 데이터 수를 줄이고자 하였고,
- 데이터를 줄일 때 최대한 해당 카테고리별와 연관이 있는 데이터를 가져오고자 함.

In [1]:
# -- 전처리 함수 -- 
import re
import pandas as pd
import string

def remove_punct(text: str) -> str:
    remove_punct_dict = dict((ord(punct), ' ') for punct in string.punctuation)
    text = text.lower().translate(remove_punct_dict)
    return text

def remove_english_and_numbers(text: str) -> str:
    text = re.sub('[a-zㄱ-ㅎ0-9]', '', text).strip()
    text = re.sub('\s{2,}', ' ', text)
    return text

def preprocess_product_name(product_name: str) -> str:
    product_name = remove_punct(product_name)
    product_name = remove_english_and_numbers(product_name)
    return product_name

# -- 전체 데이터 프레임 가져오는 함수 --
def concat_bungae_files():
    base_df = pd.DataFrame()
    for idx in range(9):
        df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')
        df = df.dropna(axis=0)
        df['cat_id'] = df['cat_id'].astype(int).astype(str)
        # -- 필요한 컬럼만 가져오기 -- 
        df = df[['product_id', 'product_name', 'cat_id']].copy()
        # -- concat 할 때마다 base_df 업데이트 -- 
        base_df = pd.concat([base_df, df], ignore_index=True)

    # -- base_df 드라이브에 저장해놓기 --
    # base_df.to_csv('/content/drive/MyDrive/bungae_base_df_fashion.csv', index=False)

    return base_df

def pull_and_preprocess_base_df():
    base_df = concat_bungae_files()
    base_df['product_name'] = base_df['product_name'].apply(lambda x : preprocess_product_name(x))
    return base_df

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/var/folders/yr/nt9qv8x55mv3g666qst4sx980000gn/T/ipykernel_68555/1555314315.py:25: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')
/var/folders/yr/nt9qv8x55mv3g666qst4sx980000gn/T/ipykernel_68555/1555314315.py:25: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')
/var/folders/yr/nt9qv8x55mv3g666qst4sx980000gn/T/ipykernel_68555/1555314315.py:25: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'./bungae_df_{idx}_fashion.csv')


In [2]:
def bring_fashion_cat_csv():
    """
    번개장터 카테고리 id와 카테고리 이름이 들어있는
    파일 가져오는 함수
    """
    final_cat = pd.read_csv('./final_category.csv')
    final_cat = final_cat.drop('Unnamed: 0', axis=1).copy()
    final_cat['cat_id'] = final_cat['cat_id'].astype(str)
    
    return final_cat

def get_cat_name(cat_id:str) -> str:
    """
    카테고리 id를 입력하면 카테고리 이름을 반환하는 함수 => 중분류에 대해서
    """
    final_cat = bring_fashion_cat_csv()
    if len(cat_id) == 6: # 중분류
        return remove_punct(list(final_cat[final_cat['cat_id'] == cat_id]['cat2'])[0])
    elif len(cat_id) == 9: # 대분류
        return remove_punct(list(final_cat[final_cat['cat_id'] == cat_id]['cat3'])[0])

# -- subs 가 있는 것들만 dictionary로 뽑아내기 --
def make_category_dict_with_subs() -> dict:
    
    with open('./bgzt_fashion_category_nums.json', 'r') as file:
        data = json.load(file)
        
    category_dict_with_subs = dict()
    for main, mids in data.items():
        for mid, subs in mids.items():
            d = defaultdict()
            if subs != [None]: # 하위 sub들이 있을 때 
                d[mid] = subs
            category_dict_with_subs.update(d)
    return category_dict_with_subs


# -- 중분류 내에서의 소분류 morphs 키워드 가져오는 함수 --
def make_subcat_morphs_by_midcat(cat_dict:dict) -> dict:
    """
    make_category_dict_with_subs 함수로 부터 받은 딕션어리로
    midcat의 하위 subcat들의 morphs 키워드 가져오는 함수 
    """
    mecab = Mecab()
    d = defaultdict(list)

    for mid, subs in cat_dict.items():
        tmp_dict = dict()
        for sub in subs:

            mid_morph = mecab.morphs(get_cat_name(mid))
            sub_morph = mecab.morphs(remove_punct(get_cat_name(sub)))
            unique_sub_morph = set(sub_morph) - set(mid_morph)
            unique_sub_morph = list(unique_sub_morph)
            if unique_sub_morph == []:
                d[mid] += sub_morph
            d[mid] += unique_sub_morph

    return dict(d)

# -- 위의 두 함수 합치기 --
def get_dict_from_midcat_with_subs():
    cat_dict = make_category_dict_with_subs()
    morphs_dict = make_subcat_morphs_by_midcat(cat_dict)
    return morphs_dict

In [3]:
from konlpy.tag import Mecab
from gensim.models.fasttext import FastText
import json
from collections import defaultdict
import gc

def get_df_by_fasttext_test_version3(cat_id:str):
    
    # -- 카테고리 id에 맞는 데이터 프레임 가져오기 --
    df = fashion_df[fashion_df['cat_id'] == cat_id]
    df = df.reset_index(drop=True)
    
    # -- tokenizing --
    mecab = Mecab()
    tokens = []
    for idx in range(len(df)): # 명사인 토큰만 가져오기 
        token = [pos[0] for pos in mecab.pos(df.loc[idx]['product_name']) if pos[1][0] == 'N']            
        if len(token) > 1:
            tokens.append(token)
            
    # -- model 1 training --       
    model1 = FastText(tokens, vector_size=100, window=10, min_count=5, workers=4, sg=0)
    model1.build_vocab(tokens)
    total_examples1 = model1.corpus_count
    model1.train(tokens, total_examples=total_examples1, epochs=10)
    
    # -- get most similar words -- 
    cat_name_morphs = mecab.morphs(get_cat_name(cat_id))
    mid_name_morphs = mecab.morphs(get_cat_name(cat_id[:6]))
    cat_name_morphs = list(set(cat_name_morphs) - set(mid_name_morphs))
    print(get_cat_name(cat_id))
    
    results = model1.wv.most_similar(positive=cat_name_morphs, topn=30)
    keywords_list = [result[0] for result in results]
    keywords = str('|'.join(keywords_list))
    
    # -- 반대되는 데이터 프레임 --
    # -- category id가 9개 자리수인 것만 소분류 내에서 다른 카테고리 키워드 들어간 데이터 삭제해주기 -- 
    cat_id_keywords_dict = get_dict_from_midcat_with_subs()
    if len(cat_id) == 9:
        keyword_list = cat_id_keywords_dict[cat_id[:6]]
        opposites = list(set(keyword_list) - set(cat_name_morphs))
        opposite_keywords = '|'.join(opposites)
        df = df[~df['product_name'].str.contains(opposite_keywords)].copy()
    
    # -- get df --
    df_filtered = df[df['product_name'].str.contains(keywords)]
    df_filtered = df_filtered.reset_index(drop=True)
    
    if len(df_filtered) > 400:
        df_filtered = df_filtered.sample(400)
    
    del model1
    gc.collect()
    
    return df_filtered

In [13]:
def reduce_data():
    
    base_df = pull_and_preprocess_base_df()
    fashion_df = base_df.copy()
    
    cat_id_list = list(fashion_df['cat_id'].unique())
    final_df = pd.DataFrame()
    for cat_id in cat_id_list:
        try:
            df = get_df_by_fasttext_test_version3(cat_id)
            final_df = pd.concat([final_df, df])
        except:
            pass
    return final_df

In [10]:
final_df = final_df.reset_index(drop=True)
final_df

,product_id,product_name,cat_id
0,223861589,노스페이스 롱패딩,310090050
1,211498059,새상품 뉴발란스 베스트 블랙 패딩,310090050
2,210612980,정품 빈폴 구스다운,310090050
3,223417908,새상품급 정품 얇은 고급 패딩 착샷 통통,310090050
4,211768829,네파 롱패딩 팝니다 가격내림,310090050
...,...,...,...
57094,174798546,방울털모자 겨울모자,400070999
57095,164895194,모자,400070999
57096,220016723,노스페이스 남성등산모자 고어텍스 여름등산모자 사이즈,400070999
57097,113371163,모자,400070999


In [33]:
from emoji import core
def remove_emojis(text):
    return core.replace_emoji(text, replace='')

# -- label encoding --
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labels = encoder.fit_transform(final_df['cat_id'])
final_df['label'] = labels

final_df['product_name'] = final_df['product_name'].apply(lambda x : remove_emojis(x))

In [39]:
final_df

,product_id,product_name,cat_id,label
0,223861589,노스페이스 롱패딩,310090050,16
1,211498059,새상품 뉴발란스 베스트 블랙 패딩,310090050,16
2,210612980,정품 빈폴 구스다운,310090050,16
3,223417908,새상품급 정품 얇은 고급 패딩 착샷 통통,310090050,16
4,211768829,네파 롱패딩 팝니다 가격내림,310090050,16
...,...,...,...,...
57094,174798546,방울털모자 겨울모자,400070999,112
57095,164895194,모자,400070999,112
57096,220016723,노스페이스 남성등산모자 고어텍스 여름등산모자 사이즈,400070999,112
57097,113371163,모자,400070999,112


In [44]:
# -- 데이터 셋 스플릿 -- 
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(final_df['product_name'],
                                                    final_df['label'].values, 
                                                    random_state=5, 
                                                    test_size=.2)
test_y

array([  9,  60, 128, ..., 104, 144,  19])

In [15]:
final_df

,product_id,product_name,cat_id,label
0,223861589,노스페이스 롱패딩,310090050,16
1,211498059,새상품 뉴발란스 베스트 블랙 패딩,310090050,16
2,210612980,정품 빈폴 구스다운,310090050,16
3,223417908,새상품급 정품 얇은 고급 패딩 착샷 통통,310090050,16
4,211768829,네파 롱패딩 팝니다 가격내림,310090050,16
...,...,...,...,...
57094,174798546,방울털모자 겨울모자,400070999,112
57095,164895194,모자,400070999,112
57096,220016723,노스페이스 남성등산모자 고어텍스 여름등산모자 사이즈,400070999,112
57097,113371163,모자,400070999,112


In [37]:
my_str = ''
for idx in range(len(final_df)):
    my_str += f"{final_df.loc[idx, 'label']},{final_df.loc[idx, 'product_name']}\n"    

In [38]:
with open('./test_fasttext_file.txt', 'w') as f:
    f.write(my_str)

In [ ]:
# -- train, validation, test 분리하고 --
# -- 모델 돌려보기 -- 